In [1]:
import numpy as np
import pandas as pd

np.random.seed(4)
n_sims = 100000

In [2]:
cov_input = pd.read_csv("test5_1.csv")

Sigma = cov_input.values
n_assets = Sigma.shape[0]
mu = np.zeros(n_assets)

simulated = np.random.multivariate_normal(
    mean=mu,
    cov=Sigma,
    size=n_sims
)

cols = cov_input.columns
sim_df = pd.DataFrame(simulated, columns=cols)

cov_output = sim_df.cov()

print("Task 5.1 - PD_input:\n", cov_output)

Task 5.1 - PD_input:
           x1        x2        x3        x4        x5
x1  0.084923  0.087932  0.042489  0.009017  0.003885
x2  0.087932  0.161030  0.058613  0.012436  0.005351
x3  0.042489  0.058613  0.037605  0.006014  0.002590
x4  0.009017  0.012436  0.006014  0.001697  0.000549
x5  0.003885  0.005351  0.002590  0.000549  0.000315


In [3]:
cov_input = pd.read_csv("test5_2.csv")

Sigma = cov_input.values
n_assets = Sigma.shape[0]
mu = np.zeros(n_assets)

simulated = np.random.multivariate_normal(
    mean=mu,
    cov=Sigma,
    size=n_sims
)

cols = cov_input.columns
sim_df = pd.DataFrame(simulated, columns=cols)

cov_output = sim_df.cov()

print("Task 5.2 - PSD_input:\n", cov_output)

Task 5.2 - PSD_input:
           x1        x2        x3        x4        x5
x1  0.084983  0.116787  0.042289  0.008973  0.003871
x2  0.116787  0.160492  0.058115  0.012332  0.005320
x3  0.042289  0.058115  0.037477  0.005955  0.002568
x4  0.008973  0.012332  0.005955  0.001687  0.000545
x5  0.003871  0.005320  0.002568  0.000545  0.000313


In [4]:
def near_psd(Sigma, eps=0.0):
    Sigma = np.asarray(Sigma, dtype=float)
    Sigma = 0.5 * (Sigma + Sigma.T)

    w, V = np.linalg.eigh(Sigma)
    w = np.clip(w, eps, None)
    Sigma_psd = V @ np.diag(w) @ V.T
    Sigma_psd = 0.5 * (Sigma_psd + Sigma_psd.T)

    return Sigma_psd

In [5]:
cov_input = pd.read_csv("test5_3.csv")

Sigma = cov_input.values
Sigma_psd = near_psd(Sigma, eps=0.0)
n_assets = Sigma_psd.shape[0]
mu = np.zeros(n_assets)

simulated = np.random.multivariate_normal(
    mean=mu,
    cov=Sigma_psd,
    size=n_sims
)

cols = cov_input.columns
sim_df = pd.DataFrame(simulated, columns=cols)

cov_output = sim_df.cov()
print("Task 5.3 - nonPSD_input_near_psd_fix:\n", cov_output)

Task 5.3 - nonPSD_input_near_psd_fix:
           x1        x2        x3        x4        x5
x1  0.085666  0.000374  0.040813  0.008185  0.003493
x2  0.000374  0.160725  0.056926  0.011762  0.005039
x3  0.040813  0.056926  0.039873  0.007449  0.003245
x4  0.008185  0.011762  0.007449  0.002548  0.000942
x5  0.003493  0.005039  0.003245  0.000942  0.000497


In [6]:
def higham_psd(Sigma, max_iter=100, tol=1e-10):
    A0 = np.asarray(Sigma, dtype=float)
    A0 = 0.5 * (A0 + A0.T)
    diag0 = np.diag(A0).copy()

    Y = A0.copy()
    deltaS = np.zeros_like(A0)

    for _ in range(max_iter):
        R = Y - deltaS
        w, V = np.linalg.eigh(R)
        w = np.clip(w, 0.0, None)
        X = V @ np.diag(w) @ V.T
        X = 0.5 * (X + X.T)

        deltaS = X - R

        Y_new = X.copy()
        np.fill_diagonal(Y_new, diag0)
        Y_new = 0.5 * (Y_new + Y_new.T)

        if np.linalg.norm(Y_new - Y, ord="fro") <= tol * max(1.0, np.linalg.norm(Y, ord="fro")):
            Y = Y_new
            break

        Y = Y_new

    w, V = np.linalg.eigh(Y)
    w = np.clip(w, 0.0, None)
    Y = V @ np.diag(w) @ V.T
    Y = 0.5 * (Y + Y.T)

    np.fill_diagonal(Y, diag0)
    Y = 0.5 * (Y + Y.T)
    return Y

In [7]:
cov_input = pd.read_csv("test5_3.csv")

Sigma = cov_input.values
Sigma_higham = higham_psd(Sigma)
n_assets = Sigma_higham.shape[0]
mu = np.zeros(n_assets)

simulated = np.random.multivariate_normal(
    mean=mu,
    cov=Sigma_higham,
    size=n_sims
)

cols = cov_input.columns
sim_df = pd.DataFrame(simulated, columns=cols)

cov_output = sim_df.cov()
print("Task 5.4 - PSD_input_higham_fix:\n", cov_output)

Task 5.4 - PSD_input_higham_fix:
           x1        x2        x3        x4        x5
x1  0.085070  0.001554  0.039609  0.008150  0.003516
x2  0.001554  0.160274  0.056100  0.011707  0.005051
x3  0.039609  0.056100  0.037589  0.007761  0.003348
x4  0.008150  0.011707  0.007761  0.001693  0.000730
x5  0.003516  0.005051  0.003348  0.000730  0.000315


In [8]:
cov_input = pd.read_csv("test5_2.csv")

Sigma = cov_input.values
Sigma = 0.5 * (Sigma + Sigma.T)
n_assets = Sigma.shape[0]
mu = np.zeros(n_assets)
eigvals, eigvecs = np.linalg.eigh(Sigma)

idx = np.argsort(eigvals)[::-1]
eigvals = eigvals[idx]
eigvecs = eigvecs[:, idx]

total = eigvals.sum()
explained = eigvals / total
cum_explained = np.cumsum(explained)

k = int(np.searchsorted(cum_explained, 0.99) + 1)

B = eigvecs[:, :k] @ np.diag(np.sqrt(eigvals[:k]))
Sigma_pca = B @ B.T

resid_diag = np.diag(Sigma - Sigma_pca)
resid_diag = np.clip(resid_diag, 0.0, None)
D_sqrt = np.diag(np.sqrt(resid_diag))

Z = np.random.normal(size=(n_sims, k))
E = np.random.normal(size=(n_sims, n_assets))

simulated = Z @ B.T + E @ D_sqrt + mu

cols = cov_input.columns
sim_df = pd.DataFrame(simulated, columns=cols)

cov_output = sim_df.cov()
print("Task 5.5 - PCA:\n", cov_output)

Task 5.5 - PCA:
           x1        x2        x3        x4        x5
x1  0.085334  0.117270  0.042381  0.008999  0.003883
x2  0.117270  0.161158  0.058243  0.012367  0.005336
x3  0.042381  0.058243  0.037369  0.006005  0.002584
x4  0.008999  0.012367  0.006005  0.001682  0.000471
x5  0.003883  0.005336  0.002584  0.000471  0.000314
